In [7]:
#General modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Text processing modules
import string
import re
import nltk
nltk.download('punkt')

# ML modules
from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [8]:
file = open('stop_words.txt', 'r')
sw=file.read().split()
file.close()
print(sw)

['беше', 'еден', 'едно', 'јас', 'рече', 'сите', 'сум', 'ние', 'сме', 'овде', 'исто', 'така', 'на', 'и', 'во', 'се', 'го', 'не', 'да', 'е', 'што', 'од', 'со', 'за', 'или', 'тоа', 'ја', 'по', 'тој', 'но', 'а', 'ги', 'до', 'таа', 'му', 'кои', 'па', 'нив', 'ни', 'ќе', 'кој', 'итн', 'бил', 'кај', 'ова', 'врз', 'над', 'тие', 'a', 'две', 'би', 'она', 'неа', 'си', 'кое', 'ако', 'два', 'има', 'в', 'како', 'само', 'дека', 'една', 'туку', 'кога', 'сега', 'ми', 'потоа', 'низ', 'ти', 'кон', 'додека', 'веќе', 'нешто', 'него', 'уште', 'таму', 'ли', 'под', 'беа', 'ме', 'некој', 'ништо', 'тука', 'пред', 'им', 'каде', 'повторно', 'ниту', 'биде', 'толку', 'никогаш', 'мене', 'дали', 'тогаш', 'своите', 'сето', 'без', 'нема', 'околу', 'многу', 'полека', 'секогаш', 'зошто', 'те', 'добро', 'можеби', 'колку', 'можеше', 'нивните', 'преку', 'миг', 'знам', 'малку', 'вратата', 'ох', 'навистина', 'оваа', 'покрај', 'повеќе', 'овој', 'сеуште', 'имаше', 'својата', 'неговите', 'неговата', 'друго', 'зашто', 'немаше', 'в

In [9]:
def clean_text(source, stop_words_list):
  # Loading the data from the source
  data = pd.read_csv(source)


  # Spliting into questions, answers and categories
  questions = data.iloc[:, 0]
  answers = data.iloc[:, 1]
  categories = data.iloc[:, 2]
  category_codes = data.iloc[:, 3]

  # Converting to questions to lower case
  questions = [q.lower() for q in questions]


  # Removing punctuation
  questions = [q.replace('--', ' ') for q in questions]
  rm_punct = str.maketrans('', '', string.punctuation+'„"')
  questions = [q.translate(rm_punct) for q in questions]


  # Split/Tokenize
  questions = [re.sub(' +', ' ', q) for q in questions]
  questions = [q.split(' ') for q in questions]


  # All-in-one
  #questions = [q.translate(rm_punct).replace('--', ' ').split(' ') for q in questions]


  # Drop stop-words
  for q in questions:
    q = [q.remove(word) for word in q[:] if word in stop_words_list]
    
  return questions, answers, categories, category_codes

In [10]:
%time
qus, ans, cats, cat_codes = clean_text('data_brainster.csv', sw)

qus[:10]

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 6.2 µs


[['кирилакадемијата',
  'дигитален',
  'маркетинг',
  'искористи',
  'ваучерот',
  'обука',
  'млади',
  'лица',
  'нуди',
  'владата'],
 ['месеци', 'недели', 'академијата', 'дигитален', 'маркетинг'],
 ['одвива', 'наставата', 'академијата', 'дигитален', 'маркетинг'],
 ['наставата', 'маркетинг', 'online'],
 ['часа', 'неделно', 'маркетинг', 'академијата'],
 ['каква', 'опрема', 'потребна', 'академијата', 'дигитален', 'маркетинг'],
 ['слушам', 'дигитален', 'маркетинг'],
 ['изучува', 'академијата', 'дигитален', 'маркетинг'],
 ['учи', 'дигитален', 'маркетинг'],
 ['слободни', 'места', 'академијата', 'дигитален', 'маркетинг']]

In [6]:
ans[:10]

0    Академијата за дигитален маркетинг трае 23 нед...
1    Академијата за дигитален маркетинг трае 23 нед...
2    Наставата за академијата за дигитален маркетин...
3    Наставата за академијата за дигитален маркетин...
4    Неделно се одржуваат три часови во времетрање ...
5    За успешно следење на наставата во рамките на ...
6    За успешно следење на наставата во рамките на ...
7    Академијата за дигитален маркетинг е предвиден...
8    Академијата за дигитален маркетинг е предвиден...
9    Повеќе информации за уписи и слободни места на...
Name: answers, dtype: object

In [ ]:
# Building the corpus

def list_to_str(list_):
  return ' '.join(list_)

corpus = [list_to_str(q) for q in qus]

corpus[:10]

['време трае академијата дигитален маркетинг',
 'месеци недели академијата дигитален маркетинг',
 'одвива наставата академијата дигитален маркетинг',
 'наставата маркетинг online',
 'часа неделно маркетинг академијата',
 'каква опрема потребна академијата дигитален маркетинг',
 'слушам дигитален маркетинг',
 'изучува академијата дигитален маркетинг',
 'учи дигитален маркетинг',
 'слободни места академијата дигитален маркетинг']

In [ ]:
# Builting the Vectorized features
count_vectorizer = CountVectorizer() #probaj bez max_features
count_vectorizer.fit(corpus)

corpus_vec = count_vectorizer.transform(corpus)

In [ ]:
print(corpus_vec[0])

  (0, 72)	1
  (0, 104)	1
  (0, 122)	1
  (0, 231)	1
  (0, 485)	1


In [ ]:
def istakniSlogotvornoR(niz):
	return re.sub(r'(^|[^aeiou])r($|[^aeiou])',r'\1R\2',niz)

re.search()

'Rzav'

In [ ]:
# Stemmer in Macedonian

to_remove = ['та']

for rem in to_remove:
  for q in qus:
    for word in q:
      if word.endswith(rem):
        print(word)
        cut_size = len(rem)
        word = word[:-cut_size]
        print(word)
#qus[:5]

академијата
академија
наставата
настава
академијата
академија
академијата
академија
академијата
академија
академијата
академија
места
мес
академијата
академија
следната
следна
академијата
академија
наставата
настава
работа
рабо
академијата
академија
академијата
академија
областа
облас
академијата
академија
цената
цена
академијата
академија
академијата
академија
академијата
академија
академијата
академија
академијата
академија
работа
рабо
академијата
академија
работата
работа
академијата
академија
академијата
академија
вашата
ваша
академијата
академија
академијата
академија
групата
група
пополнета
пополне
академијата
академија
академијата
академија
ситуацијата
ситуација
академијата
академија
академијата
академија
дипломата
диплома
академијата
академија
академијата
академија
владата
влада


In [ ]:
word = 'бравата'
rem = 'та'

if word.endswith(rem):
  cut_size = len(rem)
  word = word[:-cut_size]

print(word)

брава


1. Мали букви
2. Тргање на интерпункција
3. Токенизација
4. Тргање на stop words
5. Стеминг

In [ ]:
text = 'Денови ли се денови, „аргатски" маки големи. Како на вратот ѓердани, ниски камења стуедни, така на плешки денови легнале -та натежнале!'

In [ ]:
text_1 = str(text).translate(str.maketrans('','',string.punctuation+'„"'))
print(text_1)

Денови ли се денови аргатски маки големи Како на вратот ѓердани ниски камења стуедни така на плешки денови легнале та натежнале


In [ ]:
nltk.word_tokenize(text_1)

In [ ]:
text_2 = text_1

In [ ]:
text_2 = ['денови', 'ли', 'се', 'денови']
rem_list = ['ови']

text_3 = []

for rem in rem_list:
  print('Current rem =', rem)
  for word in text_2:
    if word.endswith(rem):
      print(f'{word} ends in {rem}')
      print(f'Was: {word}')
      word = word.replace(rem, '')
      print(f'Now is: {word}')
    text_3.append(word)

print(text_2)

print(text_3)

Current rem = ови
денови ends in ови
Was: денови
Now is: ден
денови ends in ови
Was: денови
Now is: ден
['денови', 'ли', 'се', 'денови']
['ден', 'ли', 'се', 'ден']


['Д', 'е', 'н', 'о', 'в', 'и', ' ', 'л', 'и', ' ', 'с', 'е', ' ', 'д', 'е', 'н', 'о', 'в', 'и', ' ', 'а', 'р', 'г', 'а', 'т', 'с', 'к', 'и', ' ', 'м', 'а', 'к', 'и', ' ', 'г', 'о', 'л', 'е', 'м', 'и', ' ', 'К', 'а', 'к', 'о', ' ', 'н', 'а', ' ', 'в', 'р', 'а', 'т', 'о', 'т', ' ', 'ѓ', 'е', 'р', 'д', 'а', 'н', 'и', ' ', 'н', 'и', 'с', 'к', 'и', ' ', 'к', 'а', 'м', 'е', 'њ', 'а', ' ', 'с', 'т', 'у', 'е', 'д', 'н', 'и', ' ', 'т', 'а', 'к', 'а', ' ', 'н', 'а', ' ', 'п', 'л', 'е', 'ш', 'к', 'и', ' ', 'д', 'е', 'н', 'о', 'в', 'и', ' ', 'л', 'е', 'г', 'н', 'а', 'л', 'е', ' ', 'т', 'а', ' ', 'н', 'а', 'т', 'е', 'ж', 'н', 'а', 'л', 'е']


In [ ]:
'денови'.endswith(str('ови'))

True

In [ ]:
x = 'denovi'
x = x[:-len('ovi')]

In [ ]:
x

'den'

In [ ]:
stop_words_mk = pd.read_csv('stop_words.txt', names=['Kol'])
stop_words_mk = list(stop_words_mk['Kol'])

In [ ]:
stop_words_mk[:10]

['беше', 'еден', 'едно', 'јас', 'рече', 'сите', 'сум', 'ние', 'сме', 'овде']

In [ ]:
# Функција за мали букви
def lower_case(text_input):
  return str(text_input.lower())

In [ ]:
lower_case('Денови ли се денови аргатски маки големи Како')

'денови ли се денови аргатски маки големи како'

In [ ]:
# Функција за тргање на интерпункција (бара да се вчита string)
def remove_interpuction(text_input):
  return str(text_input).translate(str.maketrans('','',string.punctuation+'„"'))

In [ ]:
remove_interpuction('ајде, да... дој4 „голем" демек! ?! Па што правите;.')

'ајде да дој4 голем демек  Па што правите'

In [ ]:
lista = [['има', 'се', 'може', 'се'],
         ['време', 'трае', 'академијата', 'графички', 'дизајн'],
 ['се', 'одвива', 'наставата', 'академијата', 'графички', 'дизајн'],
 ['каква', 'опрема', 'е', 'потребна', 'академијата', 'графички', 'дизајн'],
 ['софтвер', 'користи', 'академијата', 'графички', 'дизајн'],
 ['се', 'изучува', 'академијата', 'графички', 'дизајн'],
 ['има', 'слободни', 'места', 'академијата', 'графички', 'дизајн'],
 ['почнуваат', 'уписите', 'следната', 'академија', 'графички', 'дизајн'],
 ['постои', 'можност', 'вработување', 'завршувањето', 'академијата']]

stop_zborovi = ['се', 'има', 'е']

for l in lista:
    l = [l.remove(zbor) for zbor in l if zbor in sw]
  
print(lista)

[['се', 'се'], ['време', 'трае', 'академијата', 'графички', 'дизајн'], ['одвива', 'наставата', 'академијата', 'графички', 'дизајн'], ['каква', 'опрема', 'потребна', 'академијата', 'графички', 'дизајн'], ['софтвер', 'користи', 'академијата', 'графички', 'дизајн'], ['изучува', 'академијата', 'графички', 'дизајн'], ['слободни', 'места', 'академијата', 'графички', 'дизајн'], ['почнуваат', 'уписите', 'следната', 'академија', 'графички', 'дизајн'], ['постои', 'можност', 'вработување', 'завршувањето', 'академијата']]


In [ ]:
'се' in sw

True

In [ ]:
stop_zborovi.remove('се', 'има')

stop_zborovi

TypeError: ignored